In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [2]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

.\data\G04_FoG_trial_1_emg.csv
.\data\G04_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_1_emg.csv
.\data\G06_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_3_emg.csv
.\data\G07_Freezing_Trial1_trial_1_emg.csv
.\data\G08_FoG_1_trial_1_emg.csv
.\data\G08_FoG_2_trial_1_emg.csv
.\data\G11_FoG_trial_1_emg.csv
.\data\G11_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_3_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_3_emg.csv
.\data\P551_M050_2_A_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_2_emg.csv
.\data\P812_M050_2_B_FoG_trial_1_emg.csv
.\data\P812_M050_2_B_FoG_trial_2_emg.csv
.\data\其他\labels.txt
.\data\其他\P812_M050_2_B_FoG_trial1_annotation.csv
.\data\其他\P812_M050_2_B_FoG_trials.mat
.\data\其他\P812_M050_2_B_FoG_trial_1_out_left_foot.csv
.\data\其他\P812_M050_2_B_FoG_trial_1

In [72]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [88]:
Data = pd.read_csv('./processed data/featurePcwtf_W256_S64_dropna.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'P551_M050_2_B_FoG_trial_2_emg.csv'
ind_drop = df.columns!=drop
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin(files)
Data_sel = Data[ind]
Data_rest = Data[~ind]

In [89]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('MAV')
#ind_temp2 = feature_all.columns.str.contains('RMS')
#ind_temp3 = feature_all.columns.str.contains('VAR')
#ind_temp = ind_temp1|ind_temp2|ind_temp3
ind_temp = feature_all.columns.str.contains('mDWT')
feature = feature_all.loc[:,~ind_temp]
#feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
feature2_all = Data_rest.iloc[:,1:-1]
feature2 = feature2_all.loc[:,~ind_temp]
#feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [ ]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [211]:
import imp
imp.reload(dp)

<module 'data_processing' from 'E:\\Document\\jupyter\\Master Thesis\\data_processing.py'>

In [66]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [23]:
feature

,Label,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,...,RIGHT_BF_mDWT5,RIGHT_BF_mDWT6,RIGHT_BF_mDWT7,RIGHT_RF_mDWT0,RIGHT_RF_mDWT1,RIGHT_RF_mDWT2,RIGHT_RF_mDWT3,RIGHT_RF_mDWT4,RIGHT_RF_mDWT5,RIGHT_RF_mDWT6
0,0,540.0,454.0,748.0,620.0,1406.0,4458.0,1992.0,887.0,2070.0,...,2262.686279,1259.309937,661.646484,627.024109,422.331390,325.584045,149.540817,34.614529,15.235371,7.567358
1,0,540.0,476.0,794.0,647.0,1235.0,3225.0,2104.0,975.0,2124.0,...,6361.942383,3565.255371,1878.698730,17489.666016,17283.833984,17201.283203,17021.105469,16690.128906,10616.111328,5948.324219
2,0,525.0,477.0,731.0,624.0,1090.0,3633.0,1726.0,895.0,2051.0,...,6153.169922,3460.817871,1826.467285,962.496521,765.000916,657.255798,257.338440,152.738708,69.780663,35.270126
3,0,577.0,446.0,699.0,625.0,1108.0,5036.0,1751.0,808.0,2287.0,...,10430.298828,5854.774414,3087.293457,7725.885254,7557.866211,7455.998047,7061.154297,6838.061035,4317.638672,2414.769287
4,0,541.0,432.0,759.0,579.0,1024.0,3126.0,2008.0,906.0,2159.0,...,23542.087891,13179.492188,6941.858398,11195.717773,11015.741211,10906.076172,10106.732422,9888.559570,6314.685059,3541.675293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78514,0,1208.0,405.0,1124.0,536.0,736.0,522.0,626.0,546.0,15176.0,...,2121.493652,1188.896484,626.485840,415.748596,266.493896,197.534225,27.509951,2.184534,0.766539,0.433579
78515,0,1240.0,394.0,1108.0,529.0,753.0,540.0,626.0,567.0,15334.0,...,7.271651,4.113052,2.175889,3639.195801,3455.077148,3390.203369,3334.017578,3263.763428,2090.744141,1173.604248
78516,0,1455.0,407.0,1077.0,521.0,754.0,549.0,588.0,580.0,19859.0,...,7.267348,4.110372,2.174457,28013.242188,27784.912109,27694.750000,27228.414062,26790.941406,17023.996094,9536.416016
78517,0,1483.0,401.0,948.0,546.0,819.0,547.0,563.0,597.0,20093.0,...,10665.289062,5972.991699,3146.582275,874.318542,654.879395,566.956238,34.042343,7.987534,5.287806,2.990932


In [8]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [127]:
path = './processed data/data_set_after_window_S64_withoutSC_allPa.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]
feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

threshold_WAMP:30.0, threshold_ZC:0.0, threshold_SSC:1.0,bins:9,ranges:(-70,70)
IEMG,MAV,SSI,VAR,RMS,WL,ZC,SSC,WAMP,skew,Acti,AR,HIST


1

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [222]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [223]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [5]:
from tensorflow.keras import layers,Model,callbacks
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

In [117]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        y_01[ind1] = 1
        cw = None
    else:
        ind = ((y==1)|(y==2)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:2,1:1.,2:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=False)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.2,
                                                       random_state=555,
                                                       shuffle=True)
    
    sc = StandardScaler(with_mean=True)
    x_train = sc.fit_transform(x_train)
    x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                        epochs=200,batch_size=50,class_weight=cw,
                        callbacks=[early_stopping])
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        y_pred_v=model.predict(x_valid)
        valid = metrics.accuracy_score(y_valid,y_pred_v>0.5)
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(y_train,y_pred_ta>0.5)
    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        y_pred_v=model.predict(x_valid)
        valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1)))
        print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,valid,test,sc

def test_model(model,feature,y,sc):
    ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
    y_01 = y.copy()
    y_01[ind] = 1
    y_pred=model.predict(sc.transform(feature))
    test = metrics.accuracy_score(y_01,y_pred>0.5)
    print('acc:%f'%test)
    return test

## ANN

In [44]:
#ind = ((y==1)|(y==2)|(y==3)|(y==6))
ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y.copy()
y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature_sc,y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [119]:
input_ = layers.Input(shape=feature.shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])

In [120]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=200,batch_size=50,
                   callbacks=[early_stopping])

# Train model

In [121]:
#sc = StandardScaler(with_mean=True)
#feature_sc = sc.fit_transform(feature)
#feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,valid,test,sc = train_model(model,feature,np.array(y),True)
rest = test_model(model,feature2,np.array(y2),sc)
acc = [train,valid,test,rest]

Train on 22313 samples, validate on 5579 samples
Epoch 1/200
22313/22313 [==============================] - 2s 87us/sample - loss: 0.2870 - accuracy: 0.8884 - val_loss: 0.2010 - val_accuracy: 0.9276
Epoch 2/200
22313/22313 [==============================] - 1s 57us/sample - loss: 0.2146 - accuracy: 0.9207 - val_loss: 0.1728 - val_accuracy: 0.9349
Epoch 3/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.1848 - accuracy: 0.9307 - val_loss: 0.1607 - val_accuracy: 0.9432
Epoch 4/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.1701 - accuracy: 0.9374 - val_loss: 0.1478 - val_accuracy: 0.9482
Epoch 5/200
22313/22313 [==============================] - 1s 57us/sample - loss: 0.1575 - accuracy: 0.9406 - val_loss: 0.1377 - val_accuracy: 0.9502
Epoch 6/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.1469 - accuracy: 0.9455 - val_loss: 0.1357 - val_accuracy: 0.9496
Epoch 7/200
22313/22313 [==========================

Epoch 55/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.0276 - accuracy: 0.9899 - val_loss: 0.0568 - val_accuracy: 0.9805
Epoch 56/200
22313/22313 [==============================] - 1s 56us/sample - loss: 0.0329 - accuracy: 0.9878 - val_loss: 0.0615 - val_accuracy: 0.9812
Epoch 57/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.0303 - accuracy: 0.9892 - val_loss: 0.0613 - val_accuracy: 0.9781
Epoch 58/200
22313/22313 [==============================] - 1s 56us/sample - loss: 0.0303 - accuracy: 0.9893 - val_loss: 0.0599 - val_accuracy: 0.9796
Epoch 59/200
22313/22313 [==============================] - 1s 56us/sample - loss: 0.0282 - accuracy: 0.9897 - val_loss: 0.0703 - val_accuracy: 0.9788
Epoch 60/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.0299 - accuracy: 0.9894 - val_loss: 0.0612 - val_accuracy: 0.9803
Epoch 61/200
22313/22313 [==============================] - 1s 56us/sample - loss: 0.0248 - ac

In [68]:
test_model(model,feature2_sc,np.array(y2))

acc:0.944742


0.9447415329768271

In [39]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

array([[3498,  205,  215],
       [ 147, 6362,   61],
       [ 165,   58, 3178]], dtype=int64)

In [123]:
#acc_ann = pd.DataFrame(acc,columns=['dnn1'],index=['train','valid','test','rest data'])
#acc_ann_com=pd.concat([acc_ann_com,acc_ann.T])
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann_com.loc['dnn2',:]=acc
acc_ann_com

,train,valid,test,rest data
dnn,0.999014,0.985481,0.954402,0.942372
dnn2,0.999104,0.983689,0.949240,0.958637
dnn1,0.994756,0.977057,0.892744,0.925843


# test feature from Left or Right

In [132]:
input_ = layers.Input(shape=feature.loc[:,ind_temp].shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [126]:
acc={}
cols = ['LEFT','RIGHT']

#sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 22313 samples, validate on 5579 samples
Epoch 1/200
22313/22313 [==============================] - 2s 85us/sample - loss: 0.3701 - accuracy: 0.8440 - val_loss: 0.2692 - val_accuracy: 0.9018
Epoch 2/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.2966 - accuracy: 0.8809 - val_loss: 0.2467 - val_accuracy: 0.9075
Epoch 3/200
22313/22313 [==============================] - 2s 73us/sample - loss: 0.2734 - accuracy: 0.8926 - val_loss: 0.2421 - val_accuracy: 0.9102
Epoch 4/200
22313/22313 [==============================] - 1s 63us/sample - loss: 0.2598 - accuracy: 0.8980 - val_loss: 0.2297 - val_accuracy: 0.9147
Epoch 5/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2512 - accuracy: 0.8991 - val_loss: 0.2188 - val_accuracy: 0.9190
Epoch 6/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.2421 - accuracy: 0.9053 - val_loss: 0.2132 - val_accuracy: 0.9245
Epoch 7/200
22313/22313 [==========================

Epoch 55/200
22313/22313 [==============================] - 1s 60us/sample - loss: 0.1123 - accuracy: 0.9585 - val_loss: 0.1336 - val_accuracy: 0.9512
Epoch 56/200
22313/22313 [==============================] - 1s 57us/sample - loss: 0.1148 - accuracy: 0.9559 - val_loss: 0.1277 - val_accuracy: 0.9527
Epoch 57/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.1137 - accuracy: 0.9557 - val_loss: 0.1330 - val_accuracy: 0.9514
Epoch 58/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.1086 - accuracy: 0.9593 - val_loss: 0.1277 - val_accuracy: 0.9514
Epoch 59/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.1074 - accuracy: 0.9601 - val_loss: 0.1294 - val_accuracy: 0.9538
Epoch 60/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.1095 - accuracy: 0.9573 - val_loss: 0.1297 - val_accuracy: 0.9534
Epoch 61/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.1087 - ac

22313/22313 [==============================] - 2s 77us/sample - loss: 0.0780 - accuracy: 0.9708 - val_loss: 0.1115 - val_accuracy: 0.9606
Epoch 110/200
22313/22313 [==============================] - 2s 70us/sample - loss: 0.0820 - accuracy: 0.9688 - val_loss: 0.1086 - val_accuracy: 0.9627
test:0.905936
valid:0.963255
train:0.990051
acc:0.922966
0.9229662939555988
Train on 22313 samples, validate on 5579 samples
Epoch 1/200
22313/22313 [==============================] - 2s 74us/sample - loss: 0.4027 - accuracy: 0.8457 - val_loss: 0.2638 - val_accuracy: 0.8957
Epoch 2/200
22313/22313 [==============================] - 2s 73us/sample - loss: 0.2952 - accuracy: 0.8811 - val_loss: 0.2393 - val_accuracy: 0.9072
Epoch 3/200
22313/22313 [==============================] - 2s 70us/sample - loss: 0.2691 - accuracy: 0.8912 - val_loss: 0.2215 - val_accuracy: 0.9102
Epoch 4/200
22313/22313 [==============================] - 2s 73us/sample - loss: 0.2510 - accuracy: 0.9007 - val_loss: 0.2099 - val_ac

22313/22313 [==============================] - 1s 57us/sample - loss: 0.1072 - accuracy: 0.9571 - val_loss: 0.1217 - val_accuracy: 0.9538
Epoch 53/200
22313/22313 [==============================] - 1s 56us/sample - loss: 0.1041 - accuracy: 0.9602 - val_loss: 0.1188 - val_accuracy: 0.9557
Epoch 54/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.1095 - accuracy: 0.9581 - val_loss: 0.1155 - val_accuracy: 0.9550
Epoch 55/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.1035 - accuracy: 0.9607 - val_loss: 0.1175 - val_accuracy: 0.9532
Epoch 56/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.1004 - accuracy: 0.9613 - val_loss: 0.1197 - val_accuracy: 0.9547
Epoch 57/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.1005 - accuracy: 0.9599 - val_loss: 0.1182 - val_accuracy: 0.9541
Epoch 58/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.1018 - accuracy: 0.961

In [130]:
acc_lr=pd.DataFrame(acc,index=['train','valid','test','rest data'])#.to_csv('./results/acc_lr_ann.csv')
#acc_com=pd.concat([acc_ann_com.drop(['dnn','dnn1'],'index'),acc_lr.T])
acc_com

,train,valid,test,rest data
dnn2,0.999104,0.983689,0.949240,0.958637
LEFT,0.990051,0.963255,0.905936,0.922966
RIGHT,0.988706,0.959491,0.940493,0.931798


# test features from 2 of 8 signals

In [ ]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 22313 samples, validate on 5579 samples
Epoch 1/200
22313/22313 [==============================] - 3s 117us/sample - loss: 0.5226 - accuracy: 0.7439 - val_loss: 0.4372 - val_accuracy: 0.8021
Epoch 2/200
22313/22313 [==============================] - 1s 64us/sample - loss: 0.4659 - accuracy: 0.7791 - val_loss: 0.4173 - val_accuracy: 0.8161
Epoch 3/200
22313/22313 [==============================] - 1s 60us/sample - loss: 0.4457 - accuracy: 0.7947 - val_loss: 0.4039 - val_accuracy: 0.8211
Epoch 4/200
22313/22313 [==============================] - 2s 71us/sample - loss: 0.4299 - accuracy: 0.8041 - val_loss: 0.3901 - val_accuracy: 0.8294
Epoch 5/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.4178 - accuracy: 0.8109 - val_loss: 0.3813 - val_accuracy: 0.8319
Epoch 6/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.4092 - accuracy: 0.8169 - val_loss: 0.3732 - val_accuracy: 0.8385
Epoch 7/200
22313/22313 [=========================

Epoch 55/200
22313/22313 [==============================] - 1s 61us/sample - loss: 0.3058 - accuracy: 0.8717 - val_loss: 0.2862 - val_accuracy: 0.8830
Epoch 56/200
22313/22313 [==============================] - 1s 66us/sample - loss: 0.3012 - accuracy: 0.8729 - val_loss: 0.2886 - val_accuracy: 0.8803
Epoch 57/200
22313/22313 [==============================] - 1s 63us/sample - loss: 0.3002 - accuracy: 0.8736 - val_loss: 0.2897 - val_accuracy: 0.8815
Epoch 58/200
22313/22313 [==============================] - 1s 60us/sample - loss: 0.3030 - accuracy: 0.8702 - val_loss: 0.2894 - val_accuracy: 0.8794
Epoch 59/200
22313/22313 [==============================] - 1s 65us/sample - loss: 0.2980 - accuracy: 0.8732 - val_loss: 0.2863 - val_accuracy: 0.8824
Epoch 60/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2992 - accuracy: 0.8728 - val_loss: 0.2871 - val_accuracy: 0.8842
Epoch 61/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2977 - ac

22313/22313 [==============================] - 1s 52us/sample - loss: 0.2665 - accuracy: 0.8885 - val_loss: 0.2687 - val_accuracy: 0.8917
Epoch 110/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2677 - accuracy: 0.8895 - val_loss: 0.2687 - val_accuracy: 0.8916
Epoch 111/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2651 - accuracy: 0.8878 - val_loss: 0.2669 - val_accuracy: 0.8916
Epoch 112/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2682 - accuracy: 0.8861 - val_loss: 0.2650 - val_accuracy: 0.8934
Epoch 113/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2631 - accuracy: 0.8890 - val_loss: 0.2650 - val_accuracy: 0.8903
Epoch 114/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2656 - accuracy: 0.8904 - val_loss: 0.2647 - val_accuracy: 0.8894
Epoch 115/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.2654 - accuracy:

22313/22313 [==============================] - 1s 51us/sample - loss: 0.2289 - accuracy: 0.9103 - val_loss: 0.2259 - val_accuracy: 0.9120
Epoch 41/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2304 - accuracy: 0.9069 - val_loss: 0.2220 - val_accuracy: 0.9122
Epoch 42/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2291 - accuracy: 0.9094 - val_loss: 0.2183 - val_accuracy: 0.9145
Epoch 43/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2290 - accuracy: 0.9061 - val_loss: 0.2196 - val_accuracy: 0.9131
Epoch 44/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2263 - accuracy: 0.9086 - val_loss: 0.2186 - val_accuracy: 0.9152
Epoch 45/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2273 - accuracy: 0.9093 - val_loss: 0.2198 - val_accuracy: 0.9122
Epoch 46/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2235 - accuracy: 0.908

22313/22313 [==============================] - 1s 51us/sample - loss: 0.2619 - accuracy: 0.8914 - val_loss: 0.2583 - val_accuracy: 0.8937
Epoch 34/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2584 - accuracy: 0.8934 - val_loss: 0.2572 - val_accuracy: 0.8951
Epoch 35/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2598 - accuracy: 0.8910 - val_loss: 0.2592 - val_accuracy: 0.8916
Epoch 36/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2582 - accuracy: 0.8928 - val_loss: 0.2571 - val_accuracy: 0.8914
Epoch 37/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2579 - accuracy: 0.8936 - val_loss: 0.2581 - val_accuracy: 0.8932
Epoch 38/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2570 - accuracy: 0.8938 - val_loss: 0.2560 - val_accuracy: 0.8937
Epoch 39/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.2535 - accuracy: 0.894

...

22313/22313 [==============================] - 1s 51us/sample - loss: 0.2825 - accuracy: 0.8850 - val_loss: 0.2526 - val_accuracy: 0.8996
Epoch 15/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2788 - accuracy: 0.8850 - val_loss: 0.2508 - val_accuracy: 0.9016
Epoch 16/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2764 - accuracy: 0.8865 - val_loss: 0.2520 - val_accuracy: 0.8978
Epoch 17/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2741 - accuracy: 0.8889 - val_loss: 0.2470 - val_accuracy: 0.9009
Epoch 18/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2720 - accuracy: 0.8888 - val_loss: 0.2439 - val_accuracy: 0.9014
Epoch 19/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2668 - accuracy: 0.8924 - val_loss: 0.2439 - val_accuracy: 0.8994
Epoch 20/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2655 - accuracy: 0.892

22313/22313 [==============================] - 1s 51us/sample - loss: 0.2106 - accuracy: 0.9153 - val_loss: 0.2088 - val_accuracy: 0.9167
Epoch 69/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2099 - accuracy: 0.9157 - val_loss: 0.2079 - val_accuracy: 0.9179
Epoch 70/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.2125 - accuracy: 0.9152 - val_loss: 0.2060 - val_accuracy: 0.9184
Epoch 71/200
22313/22313 [==============================] - 1s 59us/sample - loss: 0.2063 - accuracy: 0.9179 - val_loss: 0.2066 - val_accuracy: 0.9201
Epoch 72/200
22313/22313 [==============================] - 1s 61us/sample - loss: 0.2084 - accuracy: 0.9163 - val_loss: 0.2044 - val_accuracy: 0.9192
Epoch 73/200
22313/22313 [==============================] - 1s 57us/sample - loss: 0.2053 - accuracy: 0.9185 - val_loss: 0.2026 - val_accuracy: 0.9188
Epoch 74/200
22313/22313 [==============================] - 2s 73us/sample - loss: 0.2084 - accuracy: 0.917

22313/22313 [==============================] - 1s 51us/sample - loss: 0.2417 - accuracy: 0.9033 - val_loss: 0.2314 - val_accuracy: 0.9084
Epoch 35/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2409 - accuracy: 0.9037 - val_loss: 0.2313 - val_accuracy: 0.9107
Epoch 36/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2386 - accuracy: 0.9038 - val_loss: 0.2318 - val_accuracy: 0.9077
Epoch 37/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2318 - accuracy: 0.9066 - val_loss: 0.2288 - val_accuracy: 0.9091
Epoch 38/200
22313/22313 [==============================] - 1s 50us/sample - loss: 0.2326 - accuracy: 0.9048 - val_loss: 0.2278 - val_accuracy: 0.9131
Epoch 39/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2331 - accuracy: 0.9050 - val_loss: 0.2269 - val_accuracy: 0.9086
Epoch 40/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2304 - accuracy: 0.905

22313/22313 [==============================] - 1s 51us/sample - loss: 0.2389 - accuracy: 0.9023 - val_loss: 0.2256 - val_accuracy: 0.9072
Epoch 28/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.2378 - accuracy: 0.9032 - val_loss: 0.2232 - val_accuracy: 0.9097
Epoch 29/200
22313/22313 [==============================] - 1s 65us/sample - loss: 0.2371 - accuracy: 0.9032 - val_loss: 0.2237 - val_accuracy: 0.9093
Epoch 30/200
22313/22313 [==============================] - 1s 57us/sample - loss: 0.2334 - accuracy: 0.9043 - val_loss: 0.2223 - val_accuracy: 0.9100
Epoch 31/200
22313/22313 [==============================] - 1s 59us/sample - loss: 0.2358 - accuracy: 0.9047 - val_loss: 0.2204 - val_accuracy: 0.9113
Epoch 32/200
22313/22313 [==============================] - 1s 59us/sample - loss: 0.2280 - accuracy: 0.9062 - val_loss: 0.2189 - val_accuracy: 0.9120
Epoch 33/200
22313/22313 [==============================] - 1s 52us/sample - loss: 0.2301 - accuracy: 0.906

22313/22313 [==============================] - 1s 51us/sample - loss: 0.3265 - accuracy: 0.8634 - val_loss: 0.2923 - val_accuracy: 0.8776
Epoch 6/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.3183 - accuracy: 0.8672 - val_loss: 0.2864 - val_accuracy: 0.8860
Epoch 7/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.3116 - accuracy: 0.8736 - val_loss: 0.2795 - val_accuracy: 0.8821
Epoch 8/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.3075 - accuracy: 0.8720 - val_loss: 0.2763 - val_accuracy: 0.8839
Epoch 9/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.3001 - accuracy: 0.8757 - val_loss: 0.2733 - val_accuracy: 0.8871
Epoch 10/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2966 - accuracy: 0.8811 - val_loss: 0.2710 - val_accuracy: 0.8899
Epoch 11/200
22313/22313 [==============================] - 1s 51us/sample - loss: 0.2908 - accuracy: 0.8832 - 

In [73]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

,train,valid,test,rest data
LEFT_TA_LEFT_TS,0.912545,0.881273,0.789818,0.782118
LEFT_TA_LEFT_BF,0.931636,0.906909,0.904873,0.864276
LEFT_TA_LEFT_RF,0.915045,0.884545,0.859200,0.898761
LEFT_TS_LEFT_BF,0.933955,0.901636,0.848727,0.824697
LEFT_TS_LEFT_RF,0.933045,0.884182,0.848582,0.856040
LEFT_BF_LEFT_RF,0.927545,0.898909,0.876073,0.868178
RIGHT_TA_RIGHT_TS,0.928909,0.902000,0.865164,0.850987
RIGHT_TA_RIGHT_BF,0.938591,0.906000,0.872582,0.864995
RIGHT_TA_RIGHT_RF,0.928455,0.902545,0.877818,0.857539
RIGHT_TS_RIGHT_BF,0.945182,0.906182,0.900800,0.850289


# Test on some of rest

In [219]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

acc:0.997041
acc:0.994048
acc:0.956284
acc:0.980276
acc:0.991055
acc:0.906706
acc:0.856749
acc:0.966764
[0.9970414201183432, 0.9940476190476191, 0.9562841530054644, 0.980276134122288, 0.9910554561717353, 0.9067055393586005, 0.8567493112947658, 0.9667644183773216]


In [263]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

0.9589472691255542

## RNN

In [53]:
feature_re = feature_sc.reshape((-1,feature.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(np.array(feature_re),np.array(y_01),test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [54]:
input_ = layers.Input(shape=[feature.shape[1],1])
lstm1 = layers.GRU(100)(input_)
#lstm2 = layers.LSTM(20)(lstm1)
output = layers.Dense(1,activation='sigmoid')(lstm1)
model = Model(inputs=[input_],outputs=[output])

In [55]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [56]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

Train on 22313 samples, validate on 5579 samples
Epoch 1/100
22313/22313 [==============================] - 58s 3ms/sample - loss: 0.6163 - accuracy: 0.6776 - val_loss: 0.5940 - val_accuracy: 0.6790
Epoch 2/100
22313/22313 [==============================] - 57s 3ms/sample - loss: 0.5819 - accuracy: 0.6933 - val_loss: 0.5408 - val_accuracy: 0.7333
Epoch 3/100
22313/22313 [==============================] - 63s 3ms/sample - loss: 0.5321 - accuracy: 0.7443 - val_loss: 0.5010 - val_accuracy: 0.7580
Epoch 4/100
22313/22313 [==============================] - 65s 3ms/sample - loss: 0.4911 - accuracy: 0.7655 - val_loss: 0.4638 - val_accuracy: 0.7817
Epoch 5/100
22313/22313 [==============================] - 73s 3ms/sample - loss: 0.4549 - accuracy: 0.7859 - val_loss: 0.4338 - val_accuracy: 0.8066
Epoch 6/100
22313/22313 [==============================] - 70s 3ms/sample - loss: 0.4163 - accuracy: 0.8061 - val_loss: 0.3958 - val_accuracy: 0.8199
Epoch 7/100
22313/22313 [==========================

Epoch 55/100
22313/22313 [==============================] - 70s 3ms/sample - loss: 0.1382 - accuracy: 0.9511 - val_loss: 0.2034 - val_accuracy: 0.9258
Epoch 56/100
22313/22313 [==============================] - 76s 3ms/sample - loss: 0.1359 - accuracy: 0.9522 - val_loss: 0.2154 - val_accuracy: 0.9231
Epoch 57/100
22313/22313 [==============================] - 81s 4ms/sample - loss: 0.1312 - accuracy: 0.9548 - val_loss: 0.2158 - val_accuracy: 0.9222
Epoch 58/100
22313/22313 [==============================] - 81s 4ms/sample - loss: 0.1331 - accuracy: 0.9530 - val_loss: 0.2091 - val_accuracy: 0.9267
Epoch 59/100
22313/22313 [==============================] - 70s 3ms/sample - loss: 0.1281 - accuracy: 0.9568 - val_loss: 0.2157 - val_accuracy: 0.9265
Epoch 60/100
22313/22313 [==============================] - 72s 3ms/sample - loss: 0.1336 - accuracy: 0.9535 - val_loss: 0.2027 - val_accuracy: 0.9287
Epoch 61/100
22313/22313 [==============================] - 70s 3ms/sample - loss: 0.1233 - ac

In [58]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [60]:
test_model(model,x_test,y_test)

acc:0.933037


0.9330369945511902